In [1]:
!pip install -U sentence-transformers

     |████████████████████████████████| 78 kB 594 kB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.1.0-py3-none-any.whl size=121000 sha256=ab56e153675a0414d0f7f68026f3e049838e3473b502d1c0c964e40ef95ace21
  Stored in directory: /root/.cache/pip/wheels/90/f0/bb/ed1add84da70092ea526466eadc2bfb197c4bcb8d4fa5f7bad
Successfully built sentence-transformers


In [2]:
import os

import numpy as np
import pandas as pd
import cloudpickle

from transformers import XLMRobertaModel, XLMRobertaTokenizer
import torch

from sentence_transformers import SentenceTransformer, util

2021-11-03 11:38:19.179684: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib
2021-11-03 11:38:19.179787: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
df = pd.read_csv('../input/final-image-traindataset-part0-4-count-5/final_images.csv', keep_default_na=False)

df

,image_url,count,filename,pured_filename,spaced_filename,undigit_filename,filename_lang,filename_en,section,spaced_filename_translit,ext,filename_contains_digit,undigit_filename_translation,final_filename,PREfinal_filename
0,http://upload.wikimedia.org/wikipedia/commons/...,4,Apoxyomenos Pio-Clementino Inv1185 n2,Apoxyomenos Pio Clementino Inv1185 n2,Apoxyomenos Pio Clementino Inv 1185 n 2,Apoxyomenos Pio Clementino Inv n,en,True,commons,,jpg,True,,Apoxyomenos Pio Clementino Inv n,Apoxyomenos Pio Clementino Inv n
1,http://upload.wikimedia.org/wikipedia/commons/...,4,Babybox - venkovní strana,Babybox venkovní strana,Babybox venkovní strana,Babybox venkovní strana,cs,False,commons,Babybox venkovni strana,jpg,False,Babybox outdoor party,Babybox outdoor party,Babybox outdoor party
2,http://upload.wikimedia.org/wikipedia/commons/...,4,Betania royal portrait,Betania royal portrait,Betania royal portrait,Betania royal portrait,en,True,commons,,jpg,False,,Betania royal portrait,Betania royal portrait
3,http://upload.wikimedia.org/wikipedia/commons/...,5,BlochGlassHarmonica,BlochGlassHarmonica,Bloch Glass Harmonica,Bloch Glass Harmonica,en,True,commons,,jpg,False,,Bloch Glass Harmonica,Bloch Glass Harmonica
4,http://upload.wikimedia.org/wikipedia/commons/...,4,BolivianChilePowder2,BolivianChilePowder2,Bolivian Chile Powder 2,Bolivian Chile Powder,en,True,commons,,jpg,True,,Bolivian Chile Powder,Bolivian Chile Powder
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69475,https://upload.wikimedia.org/wikipedia/ja/c/c3...,4,The five professors of Tokyo Bible Semiary,The five professors of Tokyo Bible Semiary,The five professors of Tokyo Bible Semiary,The five professors of Tokyo Bible Semiary,en,True,ja,,jpg,False,,The five professors of Tokyo Bible Semiary,The five professors of Tokyo Bible Semiary
69476,https://upload.wikimedia.org/wikipedia/ru/7/7b...,4,Кунцево1940,Кунцево1940,Кунцево 1940,Кунцево,ru,False,ru,Kuntsevo 1940,jpg,True,Kuntsevo,Kuntsevo,Kuntsevo
69477,https://upload.wikimedia.org/wikipedia/ru/9/95...,4,КомандирыПервойКоннойармии,КомандирыПервойКоннойармии,Командиры Первой Коннойармии,Командиры Первой Коннойармии,ru,False,ru,Komandiry Pervoj Konnojarmii,jpg,False,First Cavalry Commanders,First Cavalry Commanders,First Cavalry Commanders
69478,https://upload.wikimedia.org/wikipedia/ru/c/cb...,4,Politburo1934,Politburo1934,Politburo 1934,Politburo,fr,False,ru,,jpg,True,Politburo,Politburo,Politburo


### Loading Model

In [4]:
model = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2')

Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.77k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/723 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/402 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

## Retrieving Embeddings

In [5]:
filename_embeddings = model.encode(df['undigit_filename'].values)

filename_embeddings.shape

Batches:   0%|          | 0/2172 [00:00<?, ?it/s]

(69480, 768)

In [6]:
with open('undigit_filename_embeddings.data', 'wb') as file:
    cloudpickle.dump(filename_embeddings, file)